In [ ]:
import numpy as np

class ShuffledFrogLeapingAlgorithm:
    def __init__(self, fitness_function, num_frogs=30, dim=2, lower_bound=-10, upper_bound=10, 
                 num_memeplexes=5, step_size=1.0, max_iterations=50, tolerance=1e-6):
        self.fitness_function = fitness_function
        self.num_frogs = num_frogs
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.num_memeplexes = num_memeplexes
        self.step_size = step_size
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.frogs_per_memeplex = num_frogs // num_memeplexes
        
        self.population = self.initialize_population()
        self.fitness_values = self.evaluate_fitness(self.population)
        self.memeplexes, self.memeplexes_fitness = self.divide_into_memeplexes()
    
    def initialize_population(self):
        return np.random.uniform(low=self.lower_bound, high=self.upper_bound, size=(self.num_frogs, self.dim))
    
    def evaluate_fitness(self, population):
        return np.apply_along_axis(self.fitness_function, 1, population)
    
    def divide_into_memeplexes(self):
        sorted_indices = np.argsort(self.fitness_values)
        sorted_population = self.population[sorted_indices]
        sorted_fitness = self.fitness_values[sorted_indices]
        
        memeplexes = [sorted_population[i*self.frogs_per_memeplex:(i+1)*self.frogs_per_memeplex] 
                      for i in range(self.num_memeplexes)]
        memeplexes_fitness = [sorted_fitness[i*self.frogs_per_memeplex:(i+1)*self.frogs_per_memeplex] 
                              for i in range(self.num_memeplexes)]
        return memeplexes, memeplexes_fitness
    
    def local_search(self):
        for i in range(self.num_memeplexes):
            best_frog = self.memeplexes[i][0]
            worst_frog = self.memeplexes[i][-1]
            
            leap_vector = self.step_size * (best_frog - worst_frog)
            new_worst_frog = worst_frog + leap_vector
            new_worst_frog = np.clip(new_worst_frog, self.lower_bound, self.upper_bound)
            new_fitness = self.fitness_function(new_worst_frog)
            
            if new_fitness < self.memeplexes_fitness[i][-1]:
                self.memeplexes[i][-1] = new_worst_frog
                self.memeplexes_fitness[i][-1] = new_fitness
            else:
                random_frog = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                random_fitness = self.fitness_function(random_frog)
                if random_fitness < self.memeplexes_fitness[i][-1]:
                    self.memeplexes[i][-1] = random_frog
                    self.memeplexes_fitness[i][-1] = random_fitness
    
    def shuffle_frogs(self):
        combined_population = np.vstack(self.memeplexes)
        combined_fitness = np.hstack(self.memeplexes_fitness)
        
        sorted_indices = np.argsort(combined_fitness)
        self.population = combined_population[sorted_indices]
        self.fitness_values = combined_fitness[sorted_indices]
        
        self.memeplexes, self.memeplexes_fitness = self.divide_into_memeplexes()
    
    def optimize(self):
        for iteration in range(self.max_iterations):
            self.local_search()   
            self.shuffle_frogs() 
            
            self.fitness_values = self.evaluate_fitness(self.population)
            
            if iteration % 10 == 0:
                current_best_fitness = np.min(self.fitness_values)
                print(f"Iteration :{iteration + 1} Best Fitness: {current_best_fitness}")

            if current_best_fitness <= self.tolerance:
                print(f"Convergence reached at iteration {iteration + 1}")
                break

        best_solution = self.population[0]
        best_fitness = self.fitness_values[0]
        return best_solution, best_fitness

def objective_function(x):
    return np.sum(x**2)

sfla = ShuffledFrogLeapingAlgorithm(objective_function)
best_solution, best_fitness = sfla.optimize()

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Iteration :1 Best Fitness: 2.5682056748078566
Iteration :11 Best Fitness: 2.5682056748078566
Iteration :21 Best Fitness: 0.9235214351384996
Iteration :31 Best Fitness: 0.9235214351384996
Iteration :41 Best Fitness: 0.14226124047495792
Best Solution: [ 0.33976666 -0.16376769]
Best Fitness: 0.14226124047495792
